In [1]:
import re

from src.basic import PostgresDB
from src.config import DB_LAB_VM_CONFIG

In [2]:
db = PostgresDB(_config=DB_LAB_VM_CONFIG)

In [2]:
import enum


@enum.unique
class OperatorType(enum.Enum):
    BitMapHeapScan = "Bitmap Heap Scan"
    BitMapIndexScan = "Bitmap Index Scan"
    Gather = "Gather"
    Hash = "Hash"
    HashJoin = "Hash Join"
    IndexOnlyScan = "Index Only Scan"
    IndexScan = "Index Scan"
    Memorize = "Memoize"
    NestedLoop = "Nested Loop"
    PartialAggregate = "Partial Aggregate"
    SeqSCAN = "Seq Scan"
    Materialize = "Materialize"
    NotImplemented = "Not Implemented"


In [24]:
_file_name = r"C:\Users\QQ863\Documents\Projects\PycharmProjects\DeepO\data\plans_for_train\16a"
_content = None
with open(_file_name, "r", encoding="utf-8") as _f:
    _content = _f.read()

In [32]:
import rich

operators_raw = _content.split("->")
operators_raw[0]

'Aggregate  (cost=2127.43..2127.44 rows=1 width=64) (actual time=2631.815..2631.820 rows=1 loops=1)\n  '

In [191]:


class TreeNode:
    def __init__(self, parent, content, layer):
        self.parent = parent
        self.layer = layer
        self.content = content
        self.left = None
        self.right = None

    def add_child(self, child):
        assert self.left is None or self.right is None, child
        if self.left is None:
            self.left = child
        else:
            self.right = child

    def __str__(self):
        return self.content.strip()[:20] + " " + str(self.layer)


def operators_generator(_content):
    _temp = list()
    for _line in _content.split("\n"):
        if '->' in _line:
            yield '\n'.join(_temp)
            _temp = [_line]
        else:
            _temp.append(_line)
    yield '\n'.join(_temp)


cur_layer = 0
cur_blank = 0
root_node = TreeNode(None, operators_raw[0], cur_layer)
cur_node = root_node
blank2layer = {0: 0}
gene = tuple(operators_generator(_content))
for _op_raw in gene[1:]:
    _arrow_pos = _op_raw.find("->")
    if _arrow_pos > cur_blank:
        # # 达到下一层次
        cur_layer += 1
        cur_blank = _arrow_pos
        blank2layer[cur_blank] = cur_layer
        new_node = TreeNode(cur_node, _op_raw, cur_layer)
        cur_node.add_child(new_node)
        cur_node = new_node
    elif _arrow_pos == cur_blank:
        # # 一个新的节点
        new_node = TreeNode(cur_node.parent, _op_raw, cur_layer)
        cur_node.parent.add_child(new_node)
        cur_node = new_node
    else:
        cur_blank = _arrow_pos
        fallback_layer = blank2layer[cur_blank]
        for _ in range(cur_layer - fallback_layer + 1):
            cur_node = cur_node.parent
        cur_layer = fallback_layer
        new_node = TreeNode(cur_node, _op_raw, cur_layer)
        cur_node.add_child(new_node)
        cur_node = new_node

In [189]:
def pre_order(_cur_node):
    if _cur_node is not None:
        print(_cur_node)
        pre_order(_cur_node.left)
        pre_order(_cur_node.right)


pre_order(root_node)

Aggregate  (cost=212 0
->  Nested Loop  (co 1
->  Nested Loop  (co 2
->  Nested Loop  (co 3
->  Nested Loop  (co 4
->  Nested Loop  (co 5
->  Nested Loop  (co 6
->  Nested Loop  (co 7
->  Index Scan using 8
->  Bitmap Heap Scan 8
->  Bitmap Index Sca 9
->  Index Scan using 7
->  Index Scan using 6
->  Index Scan using 5
->  Index Scan using 4
->  Index Only Scan  3
->  Index Scan using 2


In [207]:
# # 先去掉

gene[0].strip().split("  ")

['Aggregate',
 '(cost=2127.43..2127.44 rows=1 width=64) (actual time=2631.815..2631.820 rows=1 loops=1)']

In [211]:
list(map(lambda _phase: _phase.strip(), gene[1].strip().split("\n")))[0].split("  ")

['->',
 'Nested Loop',
 '(cost=10.62..2126.65 rows=155 width=32) (actual time=122.410..2631.008 rows=1041 loops=1)']

In [217]:
list(map(lambda _phase: _phase.strip(), gene[10].strip().split("\n")))[0].split("  ")

['->',
 'Bitmap Index Scan on movie_keyword_idx_keywordid',
 '(cost=0.00..7.51 rows=411 width=0) (actual time=50.421..50.422 rows=43721 loops=1)']

In [252]:
# # 多重条件的解法
cond_exp = "((episode_nr >= 50) AND NOT (episode_nr < 100) AND (episode_nr < 100))"[1:-1]
list(map(lambda _s: _s[1:-1], re.sub("AND NOT|OR NOT|AND|OR", '', cond_exp).split("  ")))

['episode_nr >= 50', 'episode_nr < 100', 'episode_nr < 100']

In [262]:
res = re.findall("AND NOT|OR NOT|AND|OR", cond_exp)

['AND NOT', 'AND']

In [269]:
a = "323 IS NOT NULL"
a.replace(" IS NOT NULL", "")

'323'

In [311]:
re.sub("::text|\(|\)", "", "(country_code)::text = '[us]'::text")

"country_code = '[us]'"

In [327]:
re.match("[>=<\']", "'")

<re.Match object; span=(0, 1), match="'">